In [1]:
!git clone https://github.com/fastai/course-v3/
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd /content/course-v3/nbs/dl2/
from exp.nb_05b import *

fatal: destination path 'course-v3' already exists and is not an empty directory.
/content/course-v3/nbs/dl2


In [2]:
def get_data():
    # path = datasets.download_data(MNIST_URL, ext='.gz')
    path = '/content/mnist.pkl.gz'
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

In [3]:
import inspect; inspect.getsource(normalize)

'def normalize(x, m, s): return (x-m)/s\n'

In [3]:
x_train, y_train, x_valid, y_valid = get_data()
def normalize_to(x_train, x_valid):
    m, s = x_train.mean(), x_train.std()
    return (x_train-m)/s, (x_valid-m)/s
x_train, x_valid = normalize_to(x_train, x_valid)

train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
nh, bs = 50, 512
c = y_train.max().item()+1

In [4]:
# cnn
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
    def forward(self, x): return self.func(x)

def mnist_view(x): return x.view(-1, 1, 28, 28)

def flatten(x): return x.view(x.shape[0], -1)

def get_cnn_model(data):
    return nn.Sequential(
        Lambda(mnist_view),
        nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=2, padding=2), nn.ReLU(),
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten),
        nn.Linear(32, data.c)
    )

# cuda
class CudaCallback(Callback):
    def begin_fit(self):
        self.model = self.model.cuda()
    def begin_batch(self):
        self.run.xb, self.run.yb = self.xb.cuda(), self.yb.cuda()    

In [5]:
device = torch.device('cuda', 0)
torch.cuda.set_device(device)

In [6]:
loss_func = F.cross_entropy
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

cbfs = [Recorder,
        partial(AvgStatsCallback, accuracy),
        CudaCallback]

nfs = [8, 16, 32, 64, 64]
model = get_cnn_model(data)
opt = optim.SGD(model.parameters(), lr=0.4)
learn = Learner(model, opt, loss_func, data)
run = Runner(cb_funcs=cbfs)
run.fit(3, learn)

train: [1.63212625, tensor(0.4538, device='cuda:0')]
valid: [0.60477392578125, tensor(0.8269, device='cuda:0')]
train: [0.3539968359375, tensor(0.8923, device='cuda:0')]
valid: [0.1935548583984375, tensor(0.9436, device='cuda:0')]
train: [0.1789208984375, tensor(0.9454, device='cuda:0')]
valid: [0.15004815673828126, tensor(0.9531, device='cuda:0')]


---

- A5

In [7]:
xb = next(iter(data.train_dl))[0].view(-1, 1, 28, 28)
model = nn.Sequential(
        Lambda(mnist_view),
        nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=2, padding=2), nn.ReLU(),
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten),
        nn.Linear(32, data.c)
    )
model(xb).shape

torch.Size([512, 10])

In [8]:
flatten(model(xb)).shape

torch.Size([512, 10])

In [43]:
def conv2d(ni, nf, ks=3, stride=2):
    return nn.Sequential(
        nn.Conv2d(ni, nf, ks, stride=stride, padding=ks//2), nn.ReLU())
class BatchXTransform(Callback):
    _order = 2
    def __init__(self, tfm): self.tfms = tfm
    def begin_batch(self): self.run.xb = self.tfms(self.xb)

def view_tfms(*size):
    def _inner(x): return x.view(*((-1,)+size)) #for example, in case of mnist, get 1,28, 28 and render -1, 1, 28, 28
    return _inner

mnist_tfms = view_tfms(1, 28, 28)

def get_cnn_layers(nfs, data):
    nfs = [1] + nfs
    return [conv2d(nfs[i], nfs[i+1], 5 if i==0 else 3) for i in range(len(nfs)-1)] + [
              nn.AdaptiveAvgPool2d(1), Lambda(flatten), nn.Linear(nfs[-1], data.c)]

# we need to unpack the list since list is not a module subclass
def get_cnn_model(nfs, data):
    return nn.Sequential(*get_cnn_layers(nfs, data))

        # ipdb.set_trace()

def get_runner(model, data, lr=0.6, cbs=None, opt_func=optim.SGD, loss_func=F.cross_entropy):
    opt = opt_func(model.parameters(), lr=lr)
    learn = Learner(model, opt, loss_func, data)
    return learn, Runner(cb_funcs=listify(cbs))
    
model = get_cnn_model(nfs, data)
learn, run = get_runner(model, data, lr=0.4, cbs=cbfs)
run.fit(3, learn)

train: [2.16117265625, tensor(0.2348, device='cuda:0')]
valid: [0.92366650390625, tensor(0.7116, device='cuda:0')]
train: [0.657170234375, tensor(0.7930, device='cuda:0')]
valid: [0.25269130859375, tensor(0.9300, device='cuda:0')]
train: [0.23869947265625, tensor(0.9294, device='cuda:0')]
valid: [0.18047735595703124, tensor(0.9446, device='cuda:0')]


test AdaptiveAvgPool2d and Maxpool2d

In [48]:
def get_cnn_model(data):
    return nn.Sequential(
        Lambda(mnist_view),
        nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=2, padding=2), nn.ReLU(),
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.MaxPool2d(1),
        # Lambda(flatten),
        # nn.Linear(32, data.c)
    )

In [46]:
# nn.AdaptiveAvgPool2d(1)
xb, yb = next(iter(data.train_dl))
get_cnn_model(data)(xb).shape

torch.Size([512, 32, 1, 1])

In [49]:
# nn.Maxpool2d(1)
xb, yb = next(iter(data.train_dl))
get_cnn_model(data)(xb).shape

torch.Size([512, 32, 2, 2])

---

- A9